In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


In [4]:
import fastf1 
from fastf1 import get_session
import matplotlib.pyplot as plt
import seaborn as sns

from f1_etl import SessionConfig, DataConfig, create_safety_car_dataset

fastf1.Cache.enable_cache('E:\School Stuff\F1cache')

In [5]:
# Define a single race session
session = SessionConfig(
    year=2022,
    race="Sao Paolo Grand Prix",
    session_type="R"  # Race
)

# Configure the dataset
config = DataConfig(
    sessions=[session],
    cache_dir="E:\School Stuff\F1cache"
)

# Generate the dataset
dataset_22 = create_safety_car_dataset(
    config=config,
    window_size=100,
    prediction_horizon=10
)

print(f"Generated {dataset_22['config']['n_sequences']} sequences")
print(f"Features: {dataset_22['config']['feature_names']}")
print(f"Class distribution: {dataset_22['class_distribution']}")

Loading session: 2022 Sao Paolo Grand Prix R
Loading from cache: E:\School Stuff\F1cache\2022_Sao_Paolo_Grand_Prix_R.pkl


2025-06-29 22:13:02,210 - f1_etl - INFO - Processing 1512260 total telemetry rows
2025-06-29 22:13:02,211 - f1_etl - INFO - Grouping by: ['SessionId', 'Driver']
2025-06-29 22:13:21,735 - f1_etl - INFO - Total sequences generated: 30220
2025-06-29 22:13:21,825 - f1_etl - INFO - Generated 30220 sequences with shape (30220, 100, 9)
2025-06-29 22:13:21,843 - f1_etl - INFO - Handling missing values with strategy: forward_fill


Generated 30220 sequences
Features: ['Speed', 'RPM', 'nGear', 'Throttle', 'Brake', 'X', 'Y', 'Distance', 'DifferentialDistance']
Class distribution: {'green': 25860, 'safety_car': 3620, 'vsc': 460, 'yellow': 280}


In [6]:
y_2022 = (dataset_22['y'] == 1).astype(np.float32)
print("\nBinary label distribution:")
print(f"  Class 0 (negative): {np.sum(y_2022 == 0)} samples")
print(f"  Class 1 (positive): {np.sum(y_2022 == 1)} samples")


Binary label distribution:
  Class 0 (negative): 26600 samples
  Class 1 (positive): 3620 samples


In [7]:
# Define a single race session
session = SessionConfig(
    year=2023,
    race="Sao Paolo Grand Prix",
    session_type="R"  # Race
)

# Configure the dataset
config = DataConfig(
    sessions=[session],
    cache_dir="E:\School Stuff\F1cache"
)

# Generate the dataset
dataset_23 = create_safety_car_dataset(
    config=config,
    window_size=100,
    prediction_horizon=10
)

print(f"Generated {dataset_23['config']['n_sequences']} sequences")
print(f"Features: {dataset_23['config']['feature_names']}")
print(f"Class distribution: {dataset_23['class_distribution']}")

Loading session: 2023 Sao Paolo Grand Prix R
Loading from cache: E:\School Stuff\F1cache\2023_Sao_Paolo_Grand_Prix_R.pkl


2025-06-29 22:13:32,874 - f1_etl - INFO - Processing 1667860 total telemetry rows
2025-06-29 22:13:32,874 - f1_etl - INFO - Grouping by: ['SessionId', 'Driver']
2025-06-29 22:13:54,364 - f1_etl - INFO - Total sequences generated: 33320
2025-06-29 22:13:54,462 - f1_etl - INFO - Generated 33320 sequences with shape (33320, 100, 9)
2025-06-29 22:13:54,482 - f1_etl - INFO - No missing values detected, skipping imputation


Generated 33320 sequences
Features: ['Speed', 'RPM', 'nGear', 'Throttle', 'Brake', 'X', 'Y', 'Distance', 'DifferentialDistance']
Class distribution: {'green': 28720, 'red': 3860, 'safety_car': 500, 'yellow': 240}


In [8]:
y_2023 = (dataset_23['y'] == 2).astype(np.float32)
print("\nBinary label distribution:")
print(f"  Class 0 (negative): {np.sum(y_2023 == 0)} samples")
print(f"  Class 1 (positive): {np.sum(y_2023 == 1)} samples")


Binary label distribution:
  Class 0 (negative): 32820 samples
  Class 1 (positive): 500 samples


In [12]:
import itertools
from sklearn.model_selection import ParameterGrid
from sklearn.metrics import (
    # Core metrics for binary classification
    precision_score,
    recall_score, 
    f1_score,
    accuracy_score,
    
    # Confusion matrix
    confusion_matrix,
    
    # Threshold optimization
    precision_recall_curve,
    roc_curve,
    roc_auc_score,
    
    # Comprehensive report
    classification_report
)

# Define search spaces
HYPERPARAMETER_GRID = {
    # Model architecture
    'hidden_dim': [32, 64, 128, 256],
    'num_layers': [1, 2, 3],
    'dropout': [0.1, 0.2, 0.3, 0.5],
    
    # Training parameters
    'learning_rate': [1e-4, 5e-4, 1e-3, 5e-3],
    'batch_size': [64, 128, 256],
    'weight_decay': [0, 1e-5, 1e-4, 1e-3],
    
    # Data parameters
    'sequence_length': [25, 50, 100, 150],
    'prediction_horizon': [5, 10, 15, 20],
    
    # Class balancing
    'pos_weight_multiplier': [0.5, 1.0, 2.0, 5.0],  # Multiply calculated pos_weight
    
    # Regularization
    'gradient_clip': [None, 0.5, 1.0, 2.0]
}

# For quick testing - smaller grid
QUICK_GRID = {
    'hidden_dim': [32, 64],
    'num_layers': [1, 2],
    'learning_rate': [1e-4, 1e-3],
    'batch_size': [64, 128],
    'dropout': [0.2, 0.3]
}

MEDIUM_GRID = {
    'hidden_dim': [64, 96],     # Expand best architecture
    'num_layers': [2],
    'learning_rate': [1e-3, 5e-3],  # Full LR range
    'dropout': [0.1, 0.2, 0.3],
    'batch_size': [64, 128],              # Keep best batch sizes
    'weight_decay': [0, 1e-5, 1e-4]      # Light regularization focus
}

In [13]:
class GRUClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim, dropout=0.2):
        super(GRUClassifier, self).__init__()
        self.gru = nn.GRU(input_dim, hidden_dim, num_layers, batch_first=True, dropout=dropout)
        self.fc = nn.Linear(hidden_dim, output_dim)
        #self.sigmoid = nn.Sigmoid()  # For binary classification ##NEED TO REMOVE

    def forward(self, x):
        out, _ = self.gru(x)
        out = out[:, -1, :]  # take last time step
        out = self.fc(out)
        return out

In [14]:
class F1HyperparameterTuner:
    def __init__(self, param_grid, X_train, y_train, X_val, y_val, device='cuda'):
        self.param_grid = list(ParameterGrid(param_grid))
        self.X_train = X_train
        self.y_train = y_train
        self.X_val = X_val
        self.y_val = y_val
        self.device = device
        self.results = []
        
    def objective_function(self, params):
        """
        Train model with given parameters and return validation F1 score
        """
        try:
            # Create model
            model = GRUClassifier(
                input_dim=self.X_train.shape[2],
                hidden_dim=params['hidden_dim'],
                num_layers=params['num_layers'],
                output_dim=1,
                dropout=params['dropout']
            ).to(self.device)
            
            # Calculate class weights
            pos_count = np.sum(self.y_train)
            neg_count = len(self.y_train) - pos_count
            pos_weight = torch.tensor([neg_count / pos_count * params.get('pos_weight_multiplier', 1.0)]).to(self.device)
            
            # Setup training
            criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
            optimizer = torch.optim.Adam(
                model.parameters(), 
                lr=params['learning_rate'],
                weight_decay=params.get('weight_decay', 0)
            )
            
            # Create data loader
            train_dataset = TensorDataset(
                torch.tensor(self.X_train, dtype=torch.float32),
                torch.tensor(self.y_train, dtype=torch.float32)
            )
            train_loader = DataLoader(train_dataset, batch_size=params['batch_size'], shuffle=True)
            
            # Training loop (reduced epochs for speed)
            model.train()
            num_epochs = 10  # Reduced for hyperparameter search
            
            for epoch in range(num_epochs):
                total_loss = 0
                for X_batch, y_batch in train_loader:
                    X_batch, y_batch = X_batch.to(self.device), y_batch.to(self.device)
                    
                    optimizer.zero_grad()
                    outputs = model(X_batch)
                    loss = criterion(outputs, y_batch.unsqueeze(1))
                    loss.backward()
                    
                    # Gradient clipping if specified
                    if params.get('gradient_clip'):
                        torch.nn.utils.clip_grad_norm_(model.parameters(), params['gradient_clip'])
                    
                    optimizer.step()
                    total_loss += loss.item()
            
            # Validation
            model.eval()
            with torch.no_grad():
                X_val_tensor = torch.tensor(self.X_val, dtype=torch.float32).to(self.device)
                logits = model(X_val_tensor)
                probs = torch.sigmoid(logits).squeeze().cpu().numpy()
            
            # Find best F1 score
            from sklearn.metrics import precision_recall_curve, f1_score
            precision_vals, recall_vals, thresholds = precision_recall_curve(self.y_val, probs)
            f1_scores = 2 * (precision_vals * recall_vals) / (precision_vals + recall_vals + 1e-8)
            best_f1 = np.max(f1_scores)
            best_threshold = thresholds[np.argmax(f1_scores)]
            
            # Calculate final metrics
            pred_binary = (probs > best_threshold).astype(int)
            precision = precision_score(self.y_val, pred_binary, zero_division=0)
            recall = recall_score(self.y_val, pred_binary, zero_division=0)
            
            result = {
                'params': params,
                'f1_score': best_f1,
                'precision': precision,
                'recall': recall,
                'threshold': best_threshold,
                'val_loss': total_loss / len(train_loader)
            }
            
            return result
            
        except Exception as e:
            print(f"Error with params {params}: {e}")
            return {
                'params': params,
                'f1_score': 0,
                'precision': 0,
                'recall': 0,
                'threshold': 0.5,
                'val_loss': float('inf'),
                'error': str(e)
            }
    
    def search(self, max_trials=None):
        """
        Run hyperparameter search
        """
        param_combinations = self.param_grid[:max_trials] if max_trials else self.param_grid
        
        print(f"Starting hyperparameter search with {len(param_combinations)} combinations...")
        
        for i, params in enumerate(param_combinations):
            print(f"\nTrial {i+1}/{len(param_combinations)}")
            print(f"Testing params: {params}")
            
            result = self.objective_function(params)
            self.results.append(result)
            
            print(f"Result: F1={result['f1_score']:.3f}, P={result['precision']:.3f}, R={result['recall']:.3f}")
            
            # Clear GPU memory
            torch.cuda.empty_cache()
        
        # Find best result
        best_result = max(self.results, key=lambda x: x['f1_score'])
        print(f"\n🏆 Best result: F1={best_result['f1_score']:.3f}")
        print(f"Best params: {best_result['params']}")
        
        return self.results, best_result

# Usage
tuner = F1HyperparameterTuner(
    MEDIUM_GRID,  # Start with quick grid
    dataset_22['X'], y_2022, dataset_23['X'], y_2023,
    device=device
)

results, best_params = tuner.search(max_trials=30)  # Test 20 combinations

Starting hyperparameter search with 30 combinations...

Trial 1/30
Testing params: {'batch_size': 64, 'dropout': 0.1, 'hidden_dim': 64, 'learning_rate': 0.001, 'num_layers': 2, 'weight_decay': 0}
Result: F1=0.299, P=0.364, R=0.250

Trial 2/30
Testing params: {'batch_size': 64, 'dropout': 0.1, 'hidden_dim': 64, 'learning_rate': 0.001, 'num_layers': 2, 'weight_decay': 1e-05}
Result: F1=0.441, P=0.516, R=0.382

Trial 3/30
Testing params: {'batch_size': 64, 'dropout': 0.1, 'hidden_dim': 64, 'learning_rate': 0.001, 'num_layers': 2, 'weight_decay': 0.0001}
Result: F1=0.444, P=0.387, R=0.516

Trial 4/30
Testing params: {'batch_size': 64, 'dropout': 0.1, 'hidden_dim': 64, 'learning_rate': 0.005, 'num_layers': 2, 'weight_decay': 0}
Result: F1=0.165, P=0.109, R=0.324

Trial 5/30
Testing params: {'batch_size': 64, 'dropout': 0.1, 'hidden_dim': 64, 'learning_rate': 0.005, 'num_layers': 2, 'weight_decay': 1e-05}
Result: F1=0.438, P=0.459, R=0.416

Trial 6/30
Testing params: {'batch_size': 64, 'drop

In [2]:
torch.cuda.empty_cache()